# Chapter 09-02: 텐서 병렬화 심화 (Tensor Parallelism)

## 학습 목표
- Data Parallelism이 해결할 수 없는 **'모델 크기 > GPU 메모리'** 문제를 인식한다
- Megatron-LM의 **1D Tensor Parallelism** 원리를 Column/Row 분할 수식으로 직접 증명한다
- Forward Pass에서 **All-Reduce가 딱 1회만** 필요한 이유를 비선형함수(GeLU)의 element-wise 성질로 설명한다
- Transformer MLP와 Multi-Head Attention 양쪽에 TP를 적용하고 수학적 등가성을 코드로 검증한다

## 목차
1. [수학적 기초: 행렬 분할 원리](#1.-수학적-기초)
2. [왜 텐서 병렬화가 필요한가?](#2.-텐서-병렬화의-필요성)
3. [Column Parallel Linear 수식 증명 + 구현](#3.-Column-Parallel)
4. [Row Parallel Linear 수식 증명 + 구현](#4.-Row-Parallel)
5. [Transformer MLP 전체 TP 조합 (핵심!)](#5.-MLP-TP-조합)
6. [Multi-Head Attention의 TP 적용](#6.-Attention-TP)
7. [통신 횟수 분석 및 최적화](#7.-통신-최적화)
8. [정리 및 연습 문제](#8.-정리)

## 1. 수학적 기초

### 행렬 분할 대수법칙

**열(Column) 방향 분할**:
$$W = [W^{(0)}, W^{(1)}, \dots, W^{(N-1)}], \quad W^{(i)} \in \mathbb{R}^{d_{in} \times (d_{out}/N)}$$

$$XW = X[W^{(0)}, W^{(1)}, \dots] = [XW^{(0)}, XW^{(1)}, \dots] = [Y^{(0)}, Y^{(1)}, \dots]$$

→ 결과를 **열 방향으로 Concat**하면 원래 결과와 동일 (All-Reduce 불필요!)

**행(Row) 방향 분할**:
$$W = \begin{bmatrix} W^{(0)} \\ W^{(1)} \\ \vdots \end{bmatrix}, \quad W^{(i)} \in \mathbb{R}^{(d_{in}/N) \times d_{out}}$$

$$XW = [X^{(0)}, X^{(1)}, \dots] \begin{bmatrix} W^{(0)} \\ W^{(1)} \\ \vdots \end{bmatrix} = \sum_{i=0}^{N-1} X^{(i)} W^{(i)}$$

→ 부분곱을 **All-Reduce(SUM)**하면 원래 결과와 동일 (All-Reduce 1회 필요)

### Megatron-LM의 핵심 통찰: Column → Row 순서

$$Y = \underbrace{\text{GeLU}(X A_1)}_{\text{Column Parallel}} \cdot \underbrace{A_2}_{\text{Row Parallel}}$$

GeLU는 **element-wise** 함수: $\text{GeLU}([Y^{(0)}, Y^{(1)}]) = [\text{GeLU}(Y^{(0)}), \text{GeLU}(Y^{(1)})]$

따라서 Column Parallel 후 GeLU를 바로 적용할 수 있고, **중간 통신이 필요 없음!**

---

### 🐣 초등학생을 위한 친절 설명!

#### 📚 텐서 병렬화가 뭔가요?

> 1000페이지짜리 교과서(모델 가중치)가 있는데, 그 교과서가 너무 커서 책상(GPU) 하나에 안 올라가요!
>
> - **DP 방법**: 학생마다 교과서를 한 권씩 가짐 (1000페이지 × 학생 수) → 이미 책상에 안 올라가니 불가!  
> - **TP 방법**: 교과서를 **찢어서** (333페이지씩) 학생들에게 나눠 줌  
>   → 각자 맡은 부분을 계산하고, 답이 필요할 때만 서로 전달!

#### 🔧 Column vs Row 분할의 차이

> - **Column 분할**: 교과서의 **세로(열)** 단위로 나누기 → 각자 독립적으로 읽고 나중에 이어 붙이기
> - **Row 분할**: 교과서의 **가로(행)** 단위로 나누기 → 각자 부분 답을 구하고 모아서 더하기

### 📝 연습 문제

#### 문제 1: Column 분할 후 Concat 결과

$$X = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}, \quad W = \begin{bmatrix} 1 & 0 & 2 \\ 0 & 3 & 1 \end{bmatrix}$$

$W$를 Column 방향으로 2분할 ($N=2$):  
$W^{(0)} = ?$, $W^{(1)} = ?$, $XW^{(0)} = ?$, $XW^{(1)} = ?$, $[XW^{(0)}, XW^{(1)}] = XW$인지 확인하라.

<details>
<summary>💡 풀이 확인</summary>

$$W^{(0)} = \begin{bmatrix} 1 & 0 \\ 0 & 3 \end{bmatrix}, \quad W^{(1)} = \begin{bmatrix} 2 \\ 1 \end{bmatrix}$$

$$XW^{(0)} = \begin{bmatrix} 1 & 6 \\ 3 & 12 \end{bmatrix}, \quad XW^{(1)} = \begin{bmatrix} 4 \\ 10 \end{bmatrix}$$

$$[XW^{(0)}, XW^{(1)}] = \begin{bmatrix} 1 & 6 & 4 \\ 3 & 12 & 10 \end{bmatrix} = XW \quad \checkmark$$
</details>

#### 문제 2: 통신 최소화 설계

4-레이어 Transformer에 TP=4를 적용한다.  
각 레이어에서 MLP(All-Reduce 1회)와 Attention(All-Reduce 1회)이 있다면,  
Forward Pass 전체에서 총 All-Reduce 횟수는?

<details>
<summary>💡 풀이 확인</summary>

레이어당: MLP 1회 + Attention 1회 = **2회**  
4레이어: $4 \times 2 = \mathbf{8}$회

Backward에서도 동일하게 8회 → **총 16회** (단, 이것이 매 스텝 반복됨)
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

np.random.seed(42)

def gelu(x):
    """GeLU(x) = 0.5x·[1 + tanh(√(2/π)·(x + 0.044715x³))]"""
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))

def softmax(x, axis=-1):
    e = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e / e.sum(axis=axis, keepdims=True)

print("유틸리티 함수 정의 완료")
print(f"• GeLU 확인: gelu(0) = {gelu(0):.4f}, gelu(1) = {gelu(1):.4f}")

## 2. 텐서 병렬화의 필요성

In [ ]:
# ---------------------------------------------------
# 모델 크기별 단일 GPU 수용 가능 여부 분석
# ---------------------------------------------------

gpu_vram_GB = 80  # H100 80GB
usable_ratio = 0.7  # 70% 활용 (나머지는 activation, temp 등)

llms = [
    ('BERT-Large',     0.34,  1024,  24,  16),
    ('GPT-2 XL',       1.5,   1600,  48,  25),
    ('Llama-2 7B',     7.0,   4096,  32,  32),
    ('Llama-2 13B',    13.0,  5120,  40,  40),
    ('Llama-2 70B',    70.0,  8192,  80,  64),
    ('GPT-3 175B',     175.0, 12288, 96,  96),
    ('Megatron-Turing',530.0, 20480, 105, 128),
]

print(f"💾 GPU VRAM: {gpu_vram_GB} GB (사용 가능: {gpu_vram_GB * usable_ratio:.0f} GB)")
print(f"\n{'모델':<20} {'파라미터':>10} {'학습메모리(GB)':>15} {'단일GPU?':>10} {'최소 TP':>8}")
print("-" * 68)

for name, params_B, hidden, layers, heads in llms:
    train_mem = params_B * 16  # 16P 공식
    can_fit = train_mem <= gpu_vram_GB * usable_ratio
    min_tp = 1
    while train_mem / min_tp > gpu_vram_GB * usable_ratio:
        min_tp *= 2
    status = '✅' if can_fit else '❌'
    print(f"{name:<20} {params_B:>9.1f}B {train_mem:>14.0f} GB {status:>10} {min_tp:>8}")

print("\n→ 7B 모델부터 단일 GPU에서 학습 불가 → TP가 반드시 필요!")

## 3. Column Parallel Linear

In [ ]:
# ---------------------------------------------------
# Column Parallel Linear: 완전 구현 + 시각화
# ---------------------------------------------------

def column_parallel_linear(X, W, b, N_gpus, non_linearity=None):
    """
    W를 열(Column) 방향으로 N_gpus 등분하여 병렬 연산 수행.
    
    Forward: Y^(i) = f(X · W^(i) + b^(i)), 결과= [Y^(0), ..., Y^(N-1)] (Concat)
    통신: X는 모든 GPU에 broadcast (보통 이미 복사되어 있음)
    
    Returns:
        Y_full: 단일 GPU 결과와 동일한 출력
        shards: 각 GPU의 부분 출력 리스트
    """
    d_out = W.shape[1]
    split = d_out // N_gpus
    assert d_out % N_gpus == 0, f"d_out({d_out})이 N_gpus({N_gpus})로 나누어져야 합니다"

    shards = []
    for i in range(N_gpus):
        W_i = W[:, i*split:(i+1)*split]  # 열 분할
        b_i = b[i*split:(i+1)*split]
        Y_i = X @ W_i + b_i              # GPU i의 부분 연산
        if non_linearity:
            Y_i = non_linearity(Y_i)      # Element-wise → 각 GPU 독립 수행 가능
        shards.append(Y_i)

    Y_full = np.concatenate(shards, axis=-1)  # All-Gather (Concat)
    return Y_full, shards


# 테스트
B, S, d_in, d_out = 2, 3, 4, 8
N = 2

X   = np.random.randn(B, S, d_in)
W1  = np.random.randn(d_in, d_out)
b1  = np.random.randn(d_out)

# 단일 GPU 기준값 (GeLU 포함)
Y_single = gelu(X @ W1 + b1)  # shape: (2, 3, 8)

# Column Parallel (N=2)
Y_parallel, shards_cp = column_parallel_linear(X, W1, b1, N, non_linearity=gelu)

max_err = np.abs(Y_single - Y_parallel).max()
print(f"[Column Parallel Linear + GeLU]")
print(f"  입력 shape:        X={X.shape}, W1={W1.shape}, b1={b1.shape}")
print(f"  각 GPU shard:      {shards_cp[0].shape} × {N}개")
print(f"  Concat 결과:       {Y_parallel.shape}")
print(f"  단일 GPU 결과:     {Y_single.shape}")
print(f"  최대 오차:         {max_err:.2e}")
print(f"  수학적 등가 검증:  {'✅ 통과' if max_err < 1e-10 else '❌ 실패'}")
print()
print("핵심: GeLU는 element-wise이므로 Column 분할 후 각 GPU가 독립 수행 가능!")
print("→ All-Reduce없이 Concat만으로 원래 결과 복원")

## 4. Row Parallel Linear

In [ ]:
# ---------------------------------------------------
# Row Parallel Linear: 완전 구현
# ---------------------------------------------------

def row_parallel_linear(X_shards, W, b, N_gpus):
    """
    W를 행(Row) 방향으로 N_gpus 등분하여 병렬 연산 수행.
    
    입력: X_shards (이전 Column Parallel의 각 GPU 출력)
    Forward: Z^(i) = X^(i) · W^(i), 결과= ΣZ^(i) + b (All-Reduce SUM + bias)
    통신: 부분곱 Z^(i)를 All-Reduce(SUM) 필요 → 1회 발생
    
    Returns:
        Z_full: All-Reduce 완료된 최종 출력
        partial_results: 각 GPU의 부분곱 리스트
    """
    d_in2 = W.shape[0]
    split = d_in2 // N_gpus

    partial_results = []
    for i in range(N_gpus):
        W_i    = W[i*split:(i+1)*split, :]  # 행 분할
        X_shard = X_shards[i]               # 대응되는 입력 파티션
        Z_i    = X_shard @ W_i              # GPU i 부분곱
        partial_results.append(Z_i)

    # All-Reduce: 모든 부분곱 합산 (실제로는 NCCL All-Reduce)
    Z_full = sum(partial_results) + b
    return Z_full, partial_results


# 테스트
d_int, d_hidden = 8, 4  # intermediate(8) → hidden(4)

W2 = np.random.randn(d_int, d_hidden)
b2 = np.random.randn(d_hidden)

# 단일 GPU 기준값
Y_full_single = gelu(X @ W1 + b1)  # Column Parallel 이전 단계
Z_single = Y_full_single @ W2 + b2  # shape: (2, 3, 4)

# Row Parallel (shards_cp는 이전 Column Parallel의 각 GPU 출력)
Z_parallel, partials = row_parallel_linear(shards_cp, W2, b2, N)

max_err2 = np.abs(Z_single - Z_parallel).max()
print(f"[Row Parallel Linear]")
print(f"  입력 (각 GPU shard): {shards_cp[0].shape} × {N}개")
print(f"  W2 행 분할:          {W2.shape} → {N}개 × {W2.shape[0]//N}×{W2.shape[1]}")
print(f"  각 GPU 부분곱:       {partials[0].shape}")
print(f"  All-Reduce SUM 결과: {Z_parallel.shape}")
print(f"  단일 GPU 결과:       {Z_single.shape}")
print(f"  최대 오차:           {max_err2:.2e}")
print(f"  수학적 등가 검증:    {'✅ 통과' if max_err2 < 1e-10 else '❌ 실패'}")

## 5. MLP TP 조합 (핵심!)

Column Parallel + GeLU + Row Parallel의 조합으로 **MLP 전체를 TP로** 구현한다.  
중요한 점: **Forward에서 All-Reduce는 단 1회!**

In [ ]:
# ---------------------------------------------------
# 완전한 Transformer MLP의 Tensor Parallel 구현
# ---------------------------------------------------

def tp_mlp_forward(X, A1, b1, A2, b2, N_gpus):
    """
    Megatron-LM 방식 Tensor Parallel MLP Forward Pass.
    
    표준 MLP: Z = GeLU(X·A1 + b1)·A2 + b2
    
    TP 적용:
      - A1: Column Parallel (열 방향 분할, All-Reduce 없음)
      - GeLU: 각 GPU가 독립 수행
      - A2: Row Parallel (행 방향 분할, All-Reduce 1회)
    
    통신: Forward에서 All-Reduce **1회만** 발생!
    
    Returns:
        Z: 최종 출력 (단일 GPU와 동일)
        all_reduce_count: All-Reduce 발생 횟수
    """
    inter_dim = A1.shape[1]  # intermediate dimension
    split = inter_dim // N_gpus
    all_reduce_count = 0

    # ── stage 1: Column Parallel + GeLU ──────────────
    Y_shards = []
    for i in range(N_gpus):
        A1_i = A1[:, i*split:(i+1)*split]
        b1_i = b1[i*split:(i+1)*split]
        Y_i  = gelu(X @ A1_i + b1_i)   # 각 GPU 독립 수행, 통신 없음
        Y_shards.append(Y_i)
    # → All-Reduce 없음! Concat도 필요 없음 (다음 Row Parallel에 바로 넘김)

    # ── stage 2: Row Parallel ─────────────────────────
    Z_partials = []
    for i in range(N_gpus):
        A2_i = A2[i*split:(i+1)*split, :]
        Z_i  = Y_shards[i] @ A2_i      # 각 GPU 독립 부분곱
        Z_partials.append(Z_i)

    # ← All-Reduce: 여기서 딱 1회 발생!
    Z = sum(Z_partials) + b2
    all_reduce_count += 1

    return Z, all_reduce_count


# ── 테스트 ──────────────────────────────────────────
d_model = 4
d_ff    = 16   # FFN intermediate: 보통 4 × d_model
N       = 4    # 4개 GPU로 분할
B, S    = 2, 5

X  = np.random.randn(B, S, d_model)
A1 = np.random.randn(d_model, d_ff)
b1 = np.random.randn(d_ff)
A2 = np.random.randn(d_ff, d_model)
b2 = np.random.randn(d_model)

# 단일 GPU 기준값
Z_ref = gelu(X @ A1 + b1) @ A2 + b2

# Tensor Parallel (N=4)
Z_tp, ar_count = tp_mlp_forward(X, A1, b1, A2, b2, N)

err = np.abs(Z_ref - Z_tp).max()
print("=" * 55)
print(f"  Transformer MLP Tensor Parallel 검증")
print("=" * 55)
print(f"  설정: d_model={d_model}, d_ff={d_ff}, N_gpus={N}")
print(f"  X: {X.shape}, A1: {A1.shape}, A2: {A2.shape}")
print(f"  Forward All-Reduce 횟수: {ar_count}회")
print(f"  최대 수치 오차:          {err:.2e}")
print(f"  단일 GPU와 등가 검증:    {'✅ 완전 일치' if err < 1e-10 else '❌ 불일치'}")
print()
print("  A1 → Column Parallel: 각 GPU가 d_ff/N 차원 출력")
print("  GeLU → element-wise: 통신 없음 ✅")
print("  A2 → Row Parallel: 부분곱 합산 → All-Reduce 1회 ⚡")

## 6. Multi-Head Attention의 TP 적용

Multi-Head Attention에서도 동일한 원리를 적용하되, 분할 단위가 **head** 단위입니다.

$$\text{MHA}(X) = \text{Concat}(\text{head}_1, \dots, \text{head}_h) W^O$$

- $W^Q, W^K, W^V$: **Column Parallel** (head 단위로 분할)
- $W^O$: **Row Parallel** (All-Reduce 1회)

In [ ]:
# ---------------------------------------------------
# Multi-Head Attention Tensor Parallel 구현
# ---------------------------------------------------

def scaled_dot_product_attention(Q, K, V):
    """Scaled Dot-Product Attention: Attn(Q,K,V) = softmax(QK^T/√d_k)V"""
    d_k = Q.shape[-1]
    scores = Q @ K.transpose(0, 1, 3, 2) / np.sqrt(d_k)  # (B, heads, S, S)
    weights = softmax(scores, axis=-1)
    return weights @ V  # (B, heads, S, d_k)


def tp_multihead_attention(X, WQ, WK, WV, WO, bQ, bK, bV, bO, n_heads, N_gpus):
    """
    Tensor Parallel Multi-Head Attention.
    
    - WQ, WK, WV: Column Parallel (n_heads/N_gpus 헤드씩 각 GPU에)
    - WO: Row Parallel (Attention 출력 합산)
    - Forward All-Reduce: 1회
    """
    B, S, d_model = X.shape
    d_k = d_model // n_heads
    heads_per_gpu = n_heads // N_gpus
    d_per_gpu = heads_per_gpu * d_k

    attn_shards = []
    for i in range(N_gpus):
        # Column 분할: 각 GPU가 담당 헤드에 해당하는 Q, K, V 투영
        q_start = i * d_per_gpu
        q_end   = q_start + d_per_gpu

        WQ_i = WQ[:, q_start:q_end];  bQ_i = bQ[q_start:q_end]
        WK_i = WK[:, q_start:q_end];  bK_i = bK[q_start:q_end]
        WV_i = WV[:, q_start:q_end];  bV_i = bV[q_start:q_end]

        Q_i = (X @ WQ_i + bQ_i).reshape(B, S, heads_per_gpu, d_k).transpose(0, 2, 1, 3)
        K_i = (X @ WK_i + bK_i).reshape(B, S, heads_per_gpu, d_k).transpose(0, 2, 1, 3)
        V_i = (X @ WV_i + bV_i).reshape(B, S, heads_per_gpu, d_k).transpose(0, 2, 1, 3)

        # 각 GPU가 담당 헤드의 Attention 계산
        attn_i = scaled_dot_product_attention(Q_i, K_i, V_i)  # (B, h/N, S, d_k)
        attn_i = attn_i.transpose(0, 2, 1, 3).reshape(B, S, d_per_gpu)
        attn_shards.append(attn_i)

    # Row Parallel: 각 GPU의 Attention 출력을 WO에 곱하고 합산
    output_partials = []
    for i in range(N_gpus):
        wo_start = i * d_per_gpu
        wo_end   = wo_start + d_per_gpu
        WO_i = WO[wo_start:wo_end, :]  # Row 분할
        output_partials.append(attn_shards[i] @ WO_i)

    # All-Reduce (1회)
    output = sum(output_partials) + bO
    return output


# ── 단일 GPU 기준 구현 ────────────────────────────────
def single_gpu_mha(X, WQ, WK, WV, WO, bQ, bK, bV, bO, n_heads):
    B, S, d_model = X.shape
    d_k = d_model // n_heads
    Q = (X @ WQ + bQ).reshape(B, S, n_heads, d_k).transpose(0, 2, 1, 3)
    K = (X @ WK + bK).reshape(B, S, n_heads, d_k).transpose(0, 2, 1, 3)
    V = (X @ WV + bV).reshape(B, S, n_heads, d_k).transpose(0, 2, 1, 3)
    attn = scaled_dot_product_attention(Q, K, V)
    attn = attn.transpose(0, 2, 1, 3).reshape(B, S, d_model)
    return attn @ WO + bO


# ── 검증 ────────────────────────────────────────────
d_model = 8
n_heads = 4
N_gpus  = 2  # 헤드 수가 GPU 수의 배수여야 함
B, S    = 2, 5

WQ = np.random.randn(d_model, d_model); bQ = np.random.randn(d_model)
WK = np.random.randn(d_model, d_model); bK = np.random.randn(d_model)
WV = np.random.randn(d_model, d_model); bV = np.random.randn(d_model)
WO = np.random.randn(d_model, d_model); bO = np.random.randn(d_model)
X  = np.random.randn(B, S, d_model)

ref = single_gpu_mha(X, WQ, WK, WV, WO, bQ, bK, bV, bO, n_heads)
tp  = tp_multihead_attention(X, WQ, WK, WV, WO, bQ, bK, bV, bO, n_heads, N_gpus)

err_attn = np.abs(ref - tp).max()
print(f"[Multi-Head Attention TP 검증]")
print(f"  d_model={d_model}, n_heads={n_heads}, N_gpus={N_gpus}")
print(f"  GPU당 담당 헤드 수: {n_heads // N_gpus}")
print(f"  단일 GPU 출력: {ref.shape}")
print(f"  TP 출력:       {tp.shape}")
print(f"  최대 오차:     {err_attn:.2e}")
print(f"  검증:          {'✅ 완전 일치' if err_attn < 1e-10 else '❌ 불일치'}")

## 7. 통신 최적화

In [ ]:
# ---------------------------------------------------
# Transformer Block TP 통신 분석 시각화
# ---------------------------------------------------

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 왼쪽: 통신 횟수 스택 바 (Forward / Backward)
ax1 = axes[0]
components = ['MLP\n(Col→Row)', 'MHA\n(Col→Row)', 'Transformer\nBlock', '12-Layer\nGPT-2']
fwd = [1, 1, 2, 24]
bwd = [1, 1, 2, 24]

x = np.arange(len(components))
w = 0.35
ax1.bar(x - w/2, fwd, w, label='Forward', color='#42A5F5', alpha=0.85)
ax1.bar(x + w/2, bwd, w, label='Backward', color='#EF5350', alpha=0.85)

for xi, f, b in zip(x, fwd, bwd):
    ax1.text(xi - w/2, f + 0.3, str(f), ha='center', fontweight='bold', fontsize=10)
    ax1.text(xi + w/2, b + 0.3, str(b), ha='center', fontweight='bold', fontsize=10)

ax1.set_xticks(x); ax1.set_xticklabels(components, fontsize=10)
ax1.set_ylabel('All-Reduce 횟수')
ax1.set_title('Tensor Parallelism\nTransformer All-Reduce 통신 횟수', fontweight='bold')
ax1.legend(fontsize=10); ax1.grid(True, alpha=0.3, axis='y')

# 오른쪽: TP degree vs 통신 오버헤드 (이론)
ax2 = axes[1]
tp_degrees = np.arange(1, 33)
d_model = 4096
d_ff = 16384

# Forward 통신량 (All-Reduce): TP 높일수록 절감되는 연산량 vs 증가하는 통신 오버헤드
# 연산량 ∝ d_model × d_ff / TP (각 GPU의 부담)
compute_per_gpu = (d_model * d_ff) / tp_degrees  # 정규화
# 통신량 (All-Reduce): ∝ 2K ≈ 상수 (K: 활성화 크기)
comm_per_step = 2 * d_model * np.ones_like(tp_degrees, dtype=float)  # MLP input/output
comm_ratio = comm_per_step / (compute_per_gpu + comm_per_step) * 100

ax2.fill_between(tp_degrees, compute_per_gpu / compute_per_gpu[0] * 100,
                 0, alpha=0.3, color='#42A5F5', label='연산 절감 (↓ 좋음)')
ax2.plot(tp_degrees, comm_ratio, 'r-o', lw=2, ms=5,
         label='통신 시간 비율 (↑ 부담)')
ax2.axvline(x=8, color='green', linestyle='--', lw=2, label='통상 TP=8 (노드 내 GPU 수)')
ax2.set_xlabel('Tensor Parallel Degree')
ax2.set_ylabel('비율 (%)')
ax2.set_title('TP Degree vs 연산/통신 Trade-off\n(Llama d_model=4096)', fontweight='bold')
ax2.legend(fontsize=9); ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("결론:")
print("  1. TP를 늘릴수록 각 GPU의 연산 부담은 줄지만, 통신 비용 비율이 증가")
print("  2. NVLink(600GB/s)로 연결된 경우 TP=8(노드 내 GPU 수)이 일반적으로 최적")
print("  3. 인터노드(InfiniBand)에 TP를 배치하면 통신이 병목이 되어 역효과")

## 8. 정리

### 핵심 개념 요약

| 분할 방식 | 대상 레이어 | 입력 처리 | 출력 처리 | All-Reduce |
|-----------|------------|----------|----------|----------|
| **Column Parallel** | $A_1$, $W^Q$, $W^K$, $W^V$ | Broadcast (보통 이미 공유) | Concat | ❌ 없음 |
| **Row Parallel** | $A_2$, $W^O$ | 이전 단계 shard 바로 사용 | All-Reduce SUM | ✅ 1회 |

### 핵심 수식

$$\text{Column Parallel: } XW = X[W^{(0)}, \dots, W^{(N-1)}] = [Y^{(0)}, \dots, Y^{(N-1)}]$$

$$\text{Row Parallel: } XW = \sum_{i=0}^{N-1} X^{(i)} W^{(i)} \quad (\text{All-Reduce SUM})$$

### GeLU가 핵심인 이유
GeLU는 **element-wise** 함수 → Column Parallel 출력에 바로 적용 가능 → 중간 통신 Zero!

$$\text{GeLU}([Y^{(0)}, Y^{(1)}]) = [\text{GeLU}(Y^{(0)}), \text{GeLU}(Y^{(1)})]$$

### 다음 챕터 예고
**Chapter 09-03: 파이프라인 병렬화** — 모델을 레이어 단위로 스테이지에 배치하고, 마이크로배치 스케줄링으로 GPU 유휴 시간(Bubble)을 최소화하는 GPipe와 1F1B 알고리즘의 수학적 분석과 구현을 학습한다.